## Netflix DATASET Analysing

---

Luis Angel Garcia (2230177)

## Libraries

In [ ]:
import configparser
import pandas as pd
import mysql.connector

## MySql Config And Connection

In [10]:
config = configparser.ConfigParser()
config.read("config.txt")
db_config = {
    "host": config.get("DEFAULT", "host"),
    "port": config.getint("DEFAULT", "port"),
    "user": config.get("DEFAULT", "user"),
    "password": config.get("DEFAULT", "password")
}

conn = mysql.connector.connect(**db_config)
cursor= conn.cursor()

## Creacion de DATABASE y Conexion

In [11]:
database_name = config.get("DEFAULT", "database")
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
conn.commit()

conn.database = database_name
print(f"Base de datos '{database_name}' lista para usarse.")

Base de datos 'netflix_database' lista para usarse.


## Creacion de tabla de datos sucios

## Cerrar conexion 

In [8]:
cursor.close()
conn.close()